## Imports

In [2]:
import numpy as np
import open3d as o3d
from scipy.optimize import least_squares
from sklearn.cluster import DBSCAN

## Load and preprocess the cloud

In [3]:
point_cloud = o3d.io.read_point_cloud("filtered.pcd")

# Define the alpha value for concave hull creation, adjust based on data for tightness
alpha = 0.006

# Create the concave hull (alpha shape) from the point cloud
mesh = o3d.geometry.TriangleMesh.create_from_point_cloud_alpha_shape(point_cloud, alpha)

# Extract vertices (points) of the hull as the outer surface points
surface_points = np.asarray(mesh.vertices)

# Create a new point cloud for visualization of the hull vertices
surface_cloud = o3d.geometry.PointCloud()
surface_cloud.points = o3d.utility.Vector3dVector(surface_points)

points = np.asarray(surface_cloud.points)

# Visualize the concave hull surface points
o3d.visualization.draw_geometries([surface_cloud])

## DBSCAN

In [4]:
# Apply DBSCAN to ensure inliers are contiguous
def apply_connectivity_check(inliers, eps=0.5, min_samples=5):
    """
    Apply DBSCAN to ensure the inliers form a connected region.
    
    Parameters:
        inliers (ndarray): Nx3 array of inlier point coordinates.
        eps (float): Maximum distance between points for them to be considered in the same neighborhood.
        min_samples (int): Minimum number of points to form a cluster.
    
    Returns:
        largest_cluster (ndarray): The largest contiguous cluster of points.
    """
    if len(inliers) == 0:
        return np.array([])  # Return empty if no inliers

    db = DBSCAN(eps=eps, min_samples=min_samples).fit(inliers)
    labels = db.labels_

    # Find the largest cluster (most common label excluding noise)
    unique_labels, counts = np.unique(labels[labels >= 0], return_counts=True)
    if len(unique_labels) == 0:
        return np.array([])  # No valid clusters found
    
    largest_cluster_label = unique_labels[np.argmax(counts)]
    largest_cluster = inliers[labels == largest_cluster_label]

    return largest_cluster

## RANSAC

### Plane

In [5]:
def fit_plane(points):
    """
    Fit a plane to the given set of points using least squares.
    
    Parameters:
        points (ndarray): Nx3 array of points.
    
    Returns:
        plane_params (tuple): A tuple (a, b, c, d) for the plane equation ax + by + cz + d = 0.
    """
    centroid = np.mean(points, axis=0)
    shifted_points = points - centroid
    _, _, vh = np.linalg.svd(shifted_points)
    normal = vh[-1, :]
    d = -np.dot(normal, centroid)
    return (*normal, d)

# Function to calculate distance from points to plane
def distance_to_plane(points, plane_params):
    a, b, c, d = plane_params
    plane_normal = np.array([a, b, c])
    distances = np.abs(np.dot(points, plane_normal) + d) / np.linalg.norm(plane_normal)
    return distances

# RANSAC plane fitting with DBSCAN connectivity check
def ransac_plane(points, threshold, iterations, eps=0.01, min_samples=5):
    best_inliers = []
    best_params = None
    best_score = 0
    best_iter = 0

    for i in range(iterations):
        # Randomly select 3 points to define a candidate plane
        sample_points = points[np.random.choice(points.shape[0], 3, replace=False)]
        plane_params = fit_plane(sample_points)

        # Calculate distances and identify inliers within the threshold
        distances = distance_to_plane(points, plane_params)
        inliers = points[distances < threshold]

        # Ensure inliers form a connected cluster
        connected_inliers = apply_connectivity_check(inliers, eps=eps, min_samples=min_samples)

        # Update the best parameters if this model has more inliers
        if len(connected_inliers) > best_score:
            best_score = len(connected_inliers)
            best_inliers = connected_inliers
            best_params = plane_params
            best_iter = i

    return best_params, best_inliers

### Cylinder

In [6]:
import numpy as np
from scipy.optimize import least_squares
from sklearn.cluster import DBSCAN

# Supporting functions: Fit cylinder, distance to cylinder
def fit_cylinder(points, min_radius=None, max_radius=None):
    x0, y0, z0 = np.mean(points, axis=0)
    initial_axis = np.array([0, 0, 1])
    initial_radius = np.mean(np.linalg.norm(points - np.array([x0, y0, z0]), axis=1))
    initial_guess = [x0, y0, z0, *initial_axis, initial_radius]
    result = least_squares(residuals, initial_guess, args=(points,))
    return result.x if result.success else None

def residuals(params, points):
    x0, y0, z0, a, b, c, r = params
    axis_direction = np.array([a, b, c])
    axis_direction /= np.linalg.norm(axis_direction)
    vector_to_points = points - np.array([x0, y0, z0])
    projection_length = np.dot(vector_to_points, axis_direction)
    projection = projection_length[:, np.newaxis] * axis_direction
    perpendicular_distance = np.linalg.norm(vector_to_points - projection, axis=1)
    return perpendicular_distance - r

def distance_to_cylinder(points, cylinder_point, axis_direction, radius):
    """
    Calculate the distance of each point to a cylinder's surface defined by a point on the axis,
    the axis direction, and the radius, using the direct projection method as specified in the reference formula.
    """
    # Calculate the vector from the cylinder reference point to each point
    vector_to_points = points - cylinder_point

    # Calculate the projection of each point onto the axis
    projection = np.dot(vector_to_points, axis_direction)[:, np.newaxis] * axis_direction

    # Calculate the perpendicular (radial) distance to the axis
    radial_distance = np.linalg.norm(vector_to_points - projection, axis=1)

    # Calculate the distance to the cylinder surface by subtracting the radius
    distance_to_surface = np.abs(radial_distance - radius)
    
    return distance_to_surface

# RANSAC function for fitting a cylinder, now with DBSCAN connectivity check
def ransac_cylinder(points, threshold, iterations, eps=0.01, min_samples=5, min_radius=None, max_radius=None):
    """
    RANSAC-based cylinder fitting with DBSCAN-based connectivity check.

    Parameters:
        points (ndarray): Nx3 array of point coordinates.
        threshold (float): Distance threshold to consider a point as an inlier.
        iterations (int): Number of RANSAC iterations.
        eps (float): Maximum distance between points for connectivity in DBSCAN.
        min_samples (int): Minimum number of points for DBSCAN clusters.
        min_radius (float): Minimum allowed radius for the cylinder (optional).
        max_radius (float): Maximum allowed radius for the cylinder (optional).

    Returns:
        best_params (list): Parameters of the best-fit cylinder (center, axis, radius).
        best_inliers (ndarray): Largest connected inlier points that fit the cylinder model.
    """
    best_inliers = []
    best_params = None
    best_score = 0
    best_iter = 0

    for i in range(iterations):
        # Randomly sample 5 points to define a cylinder
        sample = points[np.random.choice(points.shape[0], 5, replace=False)]
        params = fit_cylinder(sample)
        if params is None:
            continue

        x0, y0, z0, a, b, c, r = params
        if (min_radius is not None and r < min_radius) or (max_radius is not None and r > max_radius):
            continue

        cylinder_center = np.array([x0, y0, z0])
        cylinder_axis = np.array([a, b, c])
        distances = distance_to_cylinder(points, cylinder_center, cylinder_axis, r)
        inliers = points[distances < threshold]

        if len(inliers) == 0:
            continue  # Skip if no inliers found

        # Apply DBSCAN to check for connectivity and get the largest cluster
        connected_inliers = apply_connectivity_check(inliers, eps=eps, min_samples=min_samples)

        params = cylinder_axis, cylinder_center, r

        if len(connected_inliers) > best_score:
            best_score = len(connected_inliers)
            best_inliers = connected_inliers
            best_params = params
            best_iter = i

    return best_params, best_inliers

In [7]:
def distance_to_cylinder(points, cylinder_point, axis_direction, radius):
    """
    Calculate the distance of each point to a cylinder's surface defined by a point on the axis,
    the axis direction, and the radius, consistent with the residuals calculation.
    """
    # Calculate the vector from the cylinder reference point to each point
    vector_to_points = points - cylinder_point

    # Calculate the projection of each point onto the axis
    projection_length = np.dot(vector_to_points, axis_direction)
    projection = projection_length[:, np.newaxis] * axis_direction

    # Calculate the perpendicular (radial) distance to the axis
    perpendicular_distance = np.linalg.norm(vector_to_points - projection, axis=1)

    # Calculate the distance to the cylinder surface
    distance_to_surface = perpendicular_distance - radius
    
    return distance_to_surface


In [16]:
a, b = ransac_cylinder(points, 0.002, 10000, eps=0.01, min_samples=5, min_radius=0.01, max_radius=0.025)

In [17]:
cone_cloud = o3d.geometry.PointCloud()
cone_cloud.points = o3d.utility.Vector3dVector(b)
cone_cloud.paint_uniform_color([0,1,0])

surface_cloud.paint_uniform_color([0,0,0])

#cone_mesh = create_cone_mesh(cone_params
#cone_mesh.paint_uniform_color([0.7, 1, 0.7])


o3d.visualization.draw_geometries([surface_cloud, cone_cloud])

## Visualize

In [6]:
def visualize_segments_by_model_type(segmented_objects, surface_cloud):
    """
    Visualizes the segmented objects with specific colors for planes and cylinders.
    
    Parameters:
        segmented_objects (list): List of segmented objects with 'model', 'params', and 'inliers'.
        surface_cloud (o3d.geometry.PointCloud): Original surface point cloud.
    """
    geometries = [surface_cloud]
    
    # Define colors for each model type
    color_map = {
        'plane': [1, 0, 0],  # Red for planes
        'cylinder': [0, 0, 1]  # Blue for cylinders
    }
    surface_cloud.paint_uniform_color([0,0,0])
    
    # Loop through each segmented object and assign the corresponding color based on model type
    for segment in segmented_objects:
        model_type = segment['model']
        segment_color = color_map.get(model_type, [0.5, 0.5, 0.5])  # Default grey if model type is unknown
        
        # Create point cloud for the current segment
        segment_cloud = o3d.geometry.PointCloud()
        segment_cloud.points = o3d.utility.Vector3dVector(segment['inliers'])
        segment_cloud.paint_uniform_color(segment_color)
        
        # Append the segment cloud to the geometries list
        geometries.append(segment_cloud)
        
    
    # Visualize all segments and the surface cloud
    o3d.visualization.draw_geometries(geometries, point_show_normal=True)

## Main

## Last prep

In [7]:
def cylinder_statistics(inliers, cylinder_params):
    stats = {}
    
    # Unpack cylinder parameters
    axis_direction, cylinder_center, cylinder_radius = cylinder_params

    # Calculate distances of inliers to the cylinder axis
    vector_to_points = inliers - cylinder_center
    projection_length = np.dot(vector_to_points, axis_direction)
    projection = projection_length[:, np.newaxis] * axis_direction
    perpendicular_distances = np.linalg.norm(vector_to_points - projection, axis=1)

    # Calculate the height of the cylinder
    height = projection_length.max() - projection_length.min()
    stats['height'] = height
    stats['radius'] = cylinder_radius

    # (a) Number of inliers
    stats['num_inliers'] = len(inliers)

    # (b) Absolute mean distance and standard deviation
    stats['mean_distance'] = np.mean(perpendicular_distances)
    stats['std_distance'] = np.std(perpendicular_distances)

    # Separate inliers within and outside the radius
    inliers_within_radius = inliers[perpendicular_distances <= cylinder_radius]
    inliers_outside_radius = inliers[perpendicular_distances > cylinder_radius]

    # (c) Number, average, and standard deviation of inliers within the radius
    stats['num_inliers_within_radius'] = len(inliers_within_radius)
    stats['mean_distance_within_radius'] = np.mean(perpendicular_distances[perpendicular_distances <= cylinder_radius]) if len(inliers_within_radius) > 0 else 0
    stats['std_distance_within_radius'] = np.std(perpendicular_distances[perpendicular_distances <= cylinder_radius]) if len(inliers_within_radius) > 0 else 0

    # (d) Number, average, and standard deviation of inliers outside the radius
    stats['num_inliers_outside_radius'] = len(inliers_outside_radius)
    stats['mean_distance_outside_radius'] = np.mean(perpendicular_distances[perpendicular_distances > cylinder_radius]) if len(inliers_outside_radius) > 0 else 0
    stats['std_distance_outside_radius'] = np.std(perpendicular_distances[perpendicular_distances > cylinder_radius]) if len(inliers_outside_radius) > 0 else 0

    # Centroids of inliers within and outside radius
    stats['centroid_within_radius'] = np.mean(inliers_within_radius, axis=0) if len(inliers_within_radius) > 0 else None
    stats['centroid_outside_radius'] = np.mean(inliers_outside_radius, axis=0) if len(inliers_outside_radius) > 0 else None

    return stats

In [8]:
def plane_statistics(inliers, plane_params):
    plane_info = {}
    a, b, c, d = plane_params
    plane_normal = np.array([a, b, c])

    # (1) A point on the plane (we use the centroid of the inliers here)
    point_on_plane = np.mean(inliers, axis=0)
    plane_info['point_on_plane'] = point_on_plane

    # (2) Two orthogonal vectors on the plane
    if np.allclose(plane_normal, [0, 0, 1]):
        orthogonal_vector1 = np.array([1, 0, 0])
    else:
        orthogonal_vector1 = np.cross(plane_normal, [0, 0, 1])
    orthogonal_vector1 /= np.linalg.norm(orthogonal_vector1)
    orthogonal_vector2 = np.cross(plane_normal, orthogonal_vector1)
    orthogonal_vector2 /= np.linalg.norm(orthogonal_vector2)

    plane_info['orthogonal_vector1'] = orthogonal_vector1
    plane_info['orthogonal_vector2'] = orthogonal_vector2

    # (3) Calculate distances of inliers to the plane
    distances = np.dot(inliers - point_on_plane, plane_normal) / np.linalg.norm(plane_normal)
    
    # (4) Separate inliers by side of the plane
    inliers_positive_side = inliers[distances > 0]
    inliers_negative_side = inliers[distances < 0]

    # General inlier statistics
    plane_info['num_inliers'] = len(inliers)
    plane_info['mean_distance'] = np.mean(np.abs(distances))
    plane_info['std_distance'] = np.std(np.abs(distances))

    # Statistics for inliers on the positive side of the plane
    if len(inliers_positive_side) > 0:
        plane_info['num_inliers_positive'] = len(inliers_positive_side)
        plane_info['mean_distance_positive'] = np.mean(distances[distances > 0])
        plane_info['std_distance_positive'] = np.std(distances[distances > 0])
        plane_info['centroid_positive'] = np.mean(inliers_positive_side, axis=0)
    else:
        plane_info['num_inliers_positive'] = 0
        plane_info['mean_distance_positive'] = None
        plane_info['std_distance_positive'] = None
        plane_info['centroid_positive'] = None

    # Statistics for inliers on the negative side of the plane
    if len(inliers_negative_side) > 0:
        plane_info['num_inliers_negative'] = len(inliers_negative_side)
        plane_info['mean_distance_negative'] = np.mean(distances[distances < 0])
        plane_info['std_distance_negative'] = np.std(distances[distances < 0])
        plane_info['centroid_negative'] = np.mean(inliers_negative_side, axis=0)
    else:
        plane_info['num_inliers_negative'] = 0
        plane_info['mean_distance_negative'] = None
        plane_info['std_distance_negative'] = None
        plane_info['centroid_negative'] = None

    return plane_info

## RUN

In [9]:

def ransac_main(points, plane_threshold, cylinder_threshold, iterations, num_planes, num_cylinders, 
                min_radius, max_radius, eps, min_samples):
    """
    Main function to run RANSAC for both cylinder and plane fitting on the point cloud.
    
    Parameters:
        points (ndarray): Nx3 array of point coordinates.
        plane_threshold (float): Distance threshold for plane inliers.
        cylinder_threshold (float): Distance threshold for cylinder inliers.
        iterations (int): Number of RANSAC iterations for each model.
        num_planes (int): Number of planes to find.
        num_cylinders (int): Number of cylinders to find.
        min_radius (float): Minimum allowed radius for the cylinder.
        max_radius (float): Maximum allowed radius for the cylinder.
        eps (float): DBSCAN epsilon for connectivity check.
        min_samples (int): DBSCAN minimum samples for cluster formation.

    Returns:
        segmented_objects (list): A list of dictionaries containing model parameters, statistics, and inliers for each detected object.
    """
    segmented_objects = []
    remaining_points = points.copy()
    
    planes_found = 0
    cylinders_found = 0
    
    while planes_found < num_planes or cylinders_found < num_cylinders:
        best_inliers = []
        best_params = None
        best_model = None
        best_score = 0

        # Try to find a cylinder
        if cylinders_found < num_cylinders:
            cylinder_params, cylinder_inliers = ransac_cylinder(
                remaining_points, threshold=cylinder_threshold, iterations=iterations, min_radius=min_radius, max_radius=max_radius, 
                eps=eps, min_samples=min_samples
            )
            if cylinder_params is not None and len(cylinder_inliers) > best_score:
                best_params = cylinder_params
                best_inliers = cylinder_inliers
                best_model = 'cylinder'
                best_score = len(cylinder_inliers)

        # Try to find a plane
        if planes_found < num_planes:
            plane_params, plane_inliers = ransac_plane(
                remaining_points, threshold=plane_threshold, iterations=iterations, eps=eps, min_samples=min_samples
            )
            if plane_params is not None and len(plane_inliers) > best_score:
                best_params = plane_params
                best_inliers = plane_inliers
                best_model = 'plane'
                best_score = len(plane_inliers)

        # If a model was found, store it and remove inliers
        if best_model:
            print(f"Found {best_model.capitalize()} with {best_score} inliers")
            
            # Initialize segmented object details
            segmented_object = {
                'model': best_model,
                'params': best_params,
                'inliers': best_inliers
            }

            # Get additional statistics based on model type
            if best_model == 'cylinder':
                stats = cylinder_statistics(best_inliers, best_params)
                segmented_object.update(stats)
                cylinders_found += 1
            elif best_model == 'plane':
                stats = plane_statistics(best_inliers, best_params)
                segmented_object.update(stats)
                planes_found += 1

            segmented_objects.append(segmented_object)

            # Remove inliers from remaining points
            remaining_points = np.array([pt for pt in remaining_points if pt not in best_inliers])

        # Exit if no more shapes are needed or no points left
        if len(remaining_points) == 0 or (planes_found >= num_planes and cylinders_found >= num_cylinders):
            break

    return segmented_objects

In [36]:
segments = ransac_main(points, 0.002, 0.002, 10000, 4, 3, eps=0.01, min_samples=5, min_radius=0.01, max_radius=0.03)

Found Plane with 1100 inliers
Found Plane with 535 inliers
Found Plane with 453 inliers
Found Plane with 296 inliers
Found Cylinder with 209 inliers
Found Cylinder with 344 inliers
Found Cylinder with 146 inliers


In [37]:
def visualize_segments_by_model_type(segmented_objects):
    """
    Visualizes each segmented object with a unique color in sequence and prints the color name assigned to each object.

    Parameters:
        segmented_objects (list): List of segmented objects with 'model', 'params', and 'inliers'.
    """
    geometries = []

    # Define a palette of colors with names in a specific order
    color_palette = [
        ("Red", [1, 0, 0]),
        ("Green", [0, 1, 0]),
        ("Blue", [0, 0, 1]),
        ("Yellow", [1, 1, 0]),
        ("Pink", [1, 0, 1]),
        ("Cyan", [0, 1, 1]),
        ("Grey", [0.5, 0.5, 0.5]),
        ("Orange", [1, 0.5, 0]),
        ("Purple", [0.5, 0, 0.5]),
        ("Teal", [0, 0.5, 0.5])
    ]

    # Loop through each segmented object and assign colors in sequence
    for i, segment in enumerate(segmented_objects):
        color_name, segment_color = color_palette[i % len(color_palette)]
        
        # Print out the model type, index, and assigned color name
        print(f"Segment {i}: Model = {segment['model']}, Color = {color_name}")

        # Create point cloud for the current segment
        segment_cloud = o3d.geometry.PointCloud()
        segment_cloud.points = o3d.utility.Vector3dVector(segment['inliers'])
        segment_cloud.paint_uniform_color(segment_color)

        # Append the segment cloud to the geometries list
        geometries.append(segment_cloud)

    # Visualize all segments
    o3d.visualization.draw_geometries(geometries, point_show_normal=True)

# Assuming `segmented_objects` is the output from ransac_main
visualize_segments_by_model_type(segments)

Segment 0: Model = plane, Color = Red
Segment 1: Model = plane, Color = Green
Segment 2: Model = plane, Color = Blue
Segment 3: Model = plane, Color = Yellow
Segment 4: Model = cylinder, Color = Pink
Segment 5: Model = cylinder, Color = Cyan
Segment 6: Model = cylinder, Color = Grey


In [40]:
def visualize_segments_by_model_type(segmented_objects, surface_cloud):
    """
    Visualizes the segmented objects with specific colors for planes and cylinders.
    
    Parameters:
        segmented_objects (list): List of segmented objects with 'model', 'params', and 'inliers'.
        surface_cloud (o3d.geometry.PointCloud): Original surface point cloud.
    """
    geometries = [surface_cloud]
    
    # Define colors for each model type
    color_map = {
        'plane': [1, 0, 0],  # Red for planes
        'cylinder': [0, 0, 1]  # Blue for cylinders
    }
    surface_cloud.paint_uniform_color([0,0,0])
    
    # Loop through each segmented object and assign the corresponding color based on model type
    for segment in segmented_objects:
        model_type = segment['model']
        segment_color = color_map.get(model_type, [0.5, 0.5, 0.5])  # Default grey if model type is unknown
        
        # Create point cloud for the current segment
        segment_cloud = o3d.geometry.PointCloud()
        segment_cloud.points = o3d.utility.Vector3dVector(segment['inliers'])
        segment_cloud.paint_uniform_color(segment_color)
        
        # Append the segment cloud to the geometries list
        geometries.append(segment_cloud)
    
    # Visualize all segments and the surface cloud
    o3d.visualization.draw_geometries(geometries, point_show_normal=True)

# Assuming `segmented_objects` is the output from ransac_main
# and `surface_cloud` is your original point cloud
visualize_segments_by_model_type(segments, surface_cloud)

In [65]:
import os

def save_segments_to_excel(segments, file_path):
    """
    Save segment data to an Excel file, appending if the file exists.
    
    Parameters:
        segments (list of dict): A list where each entry is a dictionary containing details of a segment.
        file_path (str): Path to the Excel file where data should be saved.
    """
    # Prepare an empty list to hold data rows
    data_rows = []

    # Iterate over each segment to extract and organize data
    for segment in segments:
        # Initialize row with common fields
        row = {
            'Model': segment.get('model'),
            'Num Inliers': segment.get('num_inliers', None),
            'Mean Distance': segment.get('mean_distance', None),
            'Std Deviation Distance': segment.get('std_distance', None),
        }
        
        # Add specific fields depending on the model type
        if segment['model'] == 'cylinder':
            row.update({
                'Cylinder Axis': segment['params'][0],
                'Cylinder Center': segment['params'][1],
                'Cylinder Radius': segment['params'][2],
                'Cylinder Height': segment.get('height', None),
                'Num Inliers within Radius': segment.get('num_inliers_within_radius', None),
                'Mean Distance within Radius': segment.get('mean_distance_within_radius', None),
                'Std Deviation within Radius': segment.get('std_distance_within_radius', None),
                'Num Inliers outside Radius': segment.get('num_inliers_outside_radius', None),
                'Mean Distance outside Radius': segment.get('mean_distance_outside_radius', None),
                'Std Deviation outside Radius': segment.get('std_distance_outside_radius', None),
                'Centroid within Radius': segment.get('centroid_within_radius', None),
                'Centroid outside Radius': segment.get('centroid_outside_radius', None)
            })
        elif segment['model'] == 'plane':
            row.update({
                'Plane Normal': segment['params'][:3],  # Normal vector
                'Plane Distance': segment['params'][3],  # Distance from origin (d in ax + by + cz + d = 0)
                'Point on Plane': segment.get('point_on_plane'),
                'Orthogonal Vector 1': segment.get('orthogonal_vector1'),
                'Orthogonal Vector 2': segment.get('orthogonal_vector2'),
                'Num Inliers Positive': segment.get('num_inliers_positive', None),
                'Mean Distance Positive': segment.get('mean_distance_positive', None),
                'Std Deviation Positive': segment.get('std_distance_positive', None),
                'Centroid Positive': segment.get('centroid_positive', None),
                'Num Inliers Negative': segment.get('num_inliers_negative', None),
                'Mean Distance Negative': segment.get('mean_distance_negative', None),
                'Std Deviation Negative': segment.get('std_distance_negative', None),
                'Centroid Negative': segment.get('centroid_negative', None)
            })

        # Append the row to the data list
        data_rows.append(row)

    # Convert to DataFrame
    new_data = pd.DataFrame(data_rows)

    # If the file already exists, read the existing data and append to it
    if os.path.exists(file_path):
        existing_data = pd.read_excel(file_path)
        combined_data = pd.concat([existing_data, new_data], ignore_index=True)
    else:
        # If the file does not exist, just use the new data
        combined_data = new_data

    # Save combined data back to the Excel file
    combined_data.to_excel(file_path, index=False)
    print(f"Data saved successfully to {file_path}")

In [29]:
def save_segments_to_excel(segments, file_path):
    """
    Save segment data to an Excel file, appending if the file exists.
    
    Parameters:
        segments (list of dict): A list where each entry is a dictionary containing details of a segment.
        file_path (str): Path to the Excel file where data should be saved.
    """
    # Prepare an empty list to hold data rows
    data_rows = []

    # Iterate over each segment to extract and organize data
    for segment in segments:
        # Initialize row with common fields
        row = {
            'Model': segment.get('model'),
            'Num Inliers': segment.get('num_inliers', None),
            'Mean Distance': segment.get('mean_distance', None),
            'Std Deviation Distance': segment.get('std_distance', None),
        }
        
        # Add specific fields depending on the model type
        if segment['model'] == 'cylinder':
            row.update({
                'Cylinder Axis': segment['params'][0],
                'Cylinder Center': segment['params'][1],
                'Cylinder Radius': segment['params'][2],
                'Cylinder Height': segment.get('height', None),
                'Num Inliers within Radius': segment.get('num_inliers_within_radius', None),
                'Mean Distance within Radius': segment.get('mean_distance_within_radius', None),
                'Std Deviation within Radius': segment.get('std_distance_within_radius', None),
                'Num Inliers outside Radius': segment.get('num_inliers_outside_radius', None),
                'Mean Distance outside Radius': segment.get('mean_distance_outside_radius', None),
                'Std Deviation outside Radius': segment.get('std_distance_outside_radius', None),
                'Centroid within Radius': segment.get('centroid_within_radius', None),
                'Centroid outside Radius': segment.get('centroid_outside_radius', None)
            })
        elif segment['model'] == 'plane':
            row.update({
                'Plane Normal': segment['params'][:3],  # Normal vector
                'Plane Distance': segment['params'][3],  # Distance from origin (d in ax + by + cz + d = 0)
                'Point on Plane': segment.get('point_on_plane'),
                'Orthogonal Vector 1': segment.get('orthogonal_vector1'),
                'Orthogonal Vector 2': segment.get('orthogonal_vector2'),
                'Num Inliers Positive': segment.get('num_inliers_positive', None),
                'Mean Distance Positive': segment.get('mean_distance_positive', None),
                'Std Deviation Positive': segment.get('std_distance_positive', None),
                'Centroid Positive': segment.get('centroid_positive', None),
                'Num Inliers Negative': segment.get('num_inliers_negative', None),
                'Mean Distance Negative': segment.get('mean_distance_negative', None),
                'Std Deviation Negative': segment.get('std_distance_negative', None),
                'Centroid Negative': segment.get('centroid_negative', None)
            })

        # Ask the user to enter a label for the Name column
        row['Name'] = input(f"Enter a name/label for the {segment['model']} with num_inliers={segment.get('num_inliers', 'N/A')}: ")

        # Append the row to the data list
        data_rows.append(row)

    # Convert to DataFrame
    new_data = pd.DataFrame(data_rows)

    # If the file already exists, read the existing data and append to it
    if os.path.exists(file_path):
        existing_data = pd.read_excel(file_path)
        combined_data = pd.concat([existing_data, new_data], ignore_index=True)
    else:
        # If the file does not exist, just use the new data
        combined_data = new_data

    # Save combined data back to the Excel file
    combined_data.to_excel(file_path, index=False)
    print(f"Data saved successfully to {file_path}")

In [28]:
segments

[{'model': 'cylinder',
  'params': (array([ 0.03171359, -0.52754064,  0.91919654]),
   array([-0.05943872,  0.52063344,  0.46584253]),
   0.029474323604558893),
  'inliers': array([[-0.08045867,  0.5513249 ,  0.36249539],
         [-0.08256719,  0.60911387,  0.31144267],
         [-0.08243363,  0.60832125,  0.31018469],
         ...,
         [-0.03036292,  0.50119215,  0.50589913],
         [-0.04452239,  0.61794013,  0.32053617],
         [-0.02991327,  0.5256989 ,  0.43097156]]),
  'height': 0.3144588886788303,
  'radius': 0.029474323604558893,
  'num_inliers': 1339,
  'mean_distance': 0.029804816382807527,
  'std_distance': 0.0022126520640904914,
  'num_inliers_within_radius': 587,
  'mean_distance_within_radius': 0.02765234032431985,
  'std_distance_within_radius': 0.001162837494410597,
  'num_inliers_outside_radius': 752,
  'mean_distance_outside_radius': 0.031485007135908946,
  'std_distance_outside_radius': 0.001105589868686531,
  'centroid_within_radius': array([-0.07145789,  

In [98]:
import pandas as pd
import os

In [132]:
save_segments_to_excel(segments, 'BasicRansac2.xlsx')

Enter a name/label for the plane with num_inliers=909:  Floor
Enter a name/label for the plane with num_inliers=553:  Back Wall
Enter a name/label for the cylinder with num_inliers=451:  Big Cylinder
Enter a name/label for the cylinder with num_inliers=480:  Middle Cylinder
Enter a name/label for the cylinder with num_inliers=384:  Fail


Data saved successfully to BasicRansac2.xlsx


In [15]:
def create_oriented_plane_mesh(plane_params, inlier_points):
    a, b, c, d = plane_params
    normal = np.array([a, b, c])

    # Project inlier points onto the plane
    projected_points = inlier_points - (np.dot(inlier_points, normal) + d).reshape(-1, 1) * normal / np.dot(normal, normal)

    # Perform PCA to find the principal directions on the plane
    mean = np.mean(projected_points, axis=0)
    centered_points = projected_points - mean
    _, _, vh = np.linalg.svd(centered_points)

    # The first two right singular vectors are the principal directions
    u = vh[0]
    v = vh[1]

    # Calculate the extent of the points along the principal directions
    u_coords = np.dot(centered_points, u)
    v_coords = np.dot(centered_points, v)
    u_range = np.max(u_coords) - np.min(u_coords)
    v_range = np.max(v_coords) - np.min(v_coords)

    # Create a grid of points on the plane
    resolution = 20
    u_grid = np.linspace(-u_range/2, u_range/2, resolution)
    v_grid = np.linspace(-v_range/2, v_range/2, resolution)
    U, V = np.meshgrid(u_grid, v_grid)

    # Calculate the 3D coordinates of the grid points
    grid_points = mean + U.reshape(-1, 1) * u + V.reshape(-1, 1) * v

    # Create vertices and triangles
    vertices = grid_points
    triangles = []
    for i in range(resolution - 1):
        for j in range(resolution - 1):
            v0 = i * resolution + j
            v1 = v0 + 1
            v2 = (i + 1) * resolution + j
            v3 = v2 + 1
            triangles.extend([[v0, v2, v1], [v1, v2, v3]])

    # Create Open3D mesh
    mesh = o3d.geometry.TriangleMesh()
    mesh.vertices = o3d.utility.Vector3dVector(vertices)
    mesh.triangles = o3d.utility.Vector3iVector(triangles)
    mesh.compute_vertex_normals()

    return mesh

In [16]:
def create_oriented_cylinder_mesh(cylinder_params, inlier_points):
    axis_direction, cylinder_center, radius = cylinder_params
    axis_direction = axis_direction / np.linalg.norm(axis_direction)  # Ensure unit direction

    # Project inliers onto the axis to determine the height range
    projections = np.dot(inlier_points - cylinder_center, axis_direction)
    min_proj, max_proj = projections.min(), projections.max()
    height = max_proj - min_proj  # Constrained height based on inliers

    # Generate mesh for the cylinder within calculated bounds
    cylinder_mesh = o3d.geometry.TriangleMesh.create_cylinder(radius=radius, height=height, resolution=20, split=4)
    cylinder_mesh.compute_vertex_normals()

    # Translate to match the inlier centroid position
    mesh_center = cylinder_center + axis_direction * (min_proj + max_proj) / 2
    cylinder_mesh.translate(mesh_center)

    # Align cylinder mesh to the calculated axis direction
    z_axis = np.array([0, 0, 1])
    rotation_axis = np.cross(z_axis, axis_direction)
    rotation_angle = np.arccos(np.clip(np.dot(z_axis, axis_direction), -1.0, 1.0))  # Clamp for numerical stability
    if np.linalg.norm(rotation_axis) > 1e-6:  # Avoid division by zero for nearly aligned vectors
        rotation_axis = rotation_axis / np.linalg.norm(rotation_axis)
        rotation_matrix = o3d.geometry.get_rotation_matrix_from_axis_angle(rotation_axis * rotation_angle)
        cylinder_mesh.rotate(rotation_matrix, center=cylinder_mesh.get_center())
    
    return cylinder_mesh

In [17]:
def create_meshes_from_segments(segmented_objects, original_cloud):
    """
    Generate and visualize meshes for each segmented object with color based on model type,
    along with the original point cloud.

    Parameters:
        segmented_objects (list): List of segmented objects from `ransac_main`, containing
                                  model type ('cylinder' or 'plane'), parameters, and inliers.
        original_cloud (np.ndarray): Original point cloud to display alongside the meshes.

    Returns:
        meshes (list): List of colored meshes for each segmented object and the original point cloud.
    """
    meshes = []

    # Convert the original point cloud to Open3D format and add it to the display list
    meshes.append(original_cloud)
    
    for segment in segmented_objects:
        model_type = segment['model']
        params = segment['params']
        inlier_points = segment['inliers']
        
        # Generate the appropriate mesh based on the model type
        if model_type == 'cylinder':
            mesh = create_oriented_cylinder_mesh(params, inlier_points)
            mesh.paint_uniform_color([0, 0, 1])  # Blue color for cylinders
        elif model_type == 'plane':
            mesh = create_oriented_plane_mesh(params, inlier_points)
            mesh.paint_uniform_color([1, 0, 0])  # Red color for planes
        else:
            continue  # Skip any other model types if present
        
        meshes.append(mesh)
    
    # Visualize all generated meshes along with the original cloud
    o3d.visualization.draw_geometries(meshes)
    
    return meshes

In [38]:
create_meshes_from_segments(segments, surface_cloud)

[PointCloud with 7232 points.,
 TriangleMesh with 400 points and 722 triangles.,
 TriangleMesh with 400 points and 722 triangles.,
 TriangleMesh with 400 points and 722 triangles.,
 TriangleMesh with 400 points and 722 triangles.,
 TriangleMesh with 102 points and 200 triangles.,
 TriangleMesh with 102 points and 200 triangles.,
 TriangleMesh with 102 points and 200 triangles.]